In [ ]:
# ## Mount google drive: If your dataset is saved on google drive
# from google.colab import drive
# drive.mount('/content/drive')

### Utils functions used in this exercise

In [ ]:
import cv2
import numpy as np
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from scipy import ndimage

In [ ]:
def displaymatches(img1, px1, py1, img2, px2, py2, Idx, Dist, N):
    # sort the matches according to their scores
    sorted_indices = np.argsort(Dist)
    sorted_dist = np.sort(Dist)

    # visualize the N best matches
    h1 = img1 
    h2 = img2

    for i in range(N):
        print('Match {}: dist={}'.format(i+1, sorted_dist[i]))
        plt.imshow(h1[:,:,::-1])
        plt.plot(px1[sorted_indices[i]], py1[sorted_indices[i]], 'ro')
        plt.text(px1[sorted_indices[i]]+5, py1[sorted_indices[i]]+14, str(i+1), color='r', fontweight='bold')
        plt.show()

        plt.imshow(h2[:,:,::-1])
        plt.plot(px2[Idx[sorted_indices[i]]], py2[Idx[sorted_indices[i]]], 'ro')
        plt.text(px2[Idx[sorted_indices[i]]]+5, py2[Idx[sorted_indices[i]]]+14, str(i+1), color='r', fontweight='bold')
        plt.show()

def non_max_suppression(imgDet, threshold):
    height, width = imgDet.shape

    imMax = np.zeros_like(imgDet)

    for i in range(1, height - 1):
        for j in range(1, width - 1):
            # check if the current pixel is maximum in its 8 neighbourhood
            if imgDet[i, j] >= imgDet[i-1:i+2, j-1:j+2].max() and \
               imgDet[i, j] > threshold:
                imMax[i, j] = imgDet[i, j]

    coordinates = np.where(imMax!=0)
    px, py = coordinates[1], coordinates[0]

    return px, py

## Question2: Matching

### Harris corner response function

In [ ]:
# Compute Harris corner response function
def gauss(x, sigma):
    G = #TODO: compute the gaussian(mean=0, sig=sigma) for x and put the results in "G" variable
    return G

def gaussdx(x, sigma):
    D = #TODO: compute the derivative of gaussian(mean=0, sig=sigma) for x and put the results in "D" variable
    return D

def compute_harris(img, sigma1, sigma2):
    ResImage = # TODO: Use the one from ex1.
    return ResImage


### r/g color histogram descriptor

In [ ]:
def histrg(img, bins):
    # define a 2D histogram with "bins^2" number of entries
    h = np.zeros((bins,bins))
    
    # execute the loop for each pixel in the image
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            
            # increment a histogram bin which corresponds to the value 
            # of pixel i,j; h(r,g)
            R = img[i,j,0]
            G = img[i,j,1]
            B = img[i,j,2]
            
            r = int(np.floor(R / (R + G + B + 1e-10) * bins)) + 1
            g = int(np.floor(G / (R + G + B + 1e-10) * bins)) + 1
            if r == bins + 1:
                r = bins
            if g == bins + 1:
                g = bins
                
            h[r-1, g-1] += 1

    # normalize the histogram such that its integral (sum) is equal 1
    h = h / np.sum(h)
    h = h.reshape(-1)

    return h

def descriptors_rg(img, px, py, m, bins):
    img = img.astype(np.float32)
    rad = int((m-1)/2)
    h, w, c = img.shape
    D = np.zeros((len(px), bins**2))

    for i in range(len(px)):
        minx = max(px[i]-rad, 0)
        maxx = min(px[i]+rad+1, w)
        miny = max(py[i]-rad, 0)
        maxy = min(py[i]+rad+1, h)

        imgWin = img[miny:maxy, minx:maxx, :]
        hist = histrg(imgWin, bins)
        D[i, :] = hist.flatten()

    return D

### dx/dy histogram descriptor

In [ ]:
from scipy.signal import convolve2d

def histdxdy(img, sigma, bins):
    # compute the first derivatives
    x = np.arange(np.floor(-3.0 * sigma + 0.5), np.floor(3.0 * sigma + 0.5) + 1)
    G = gauss(x, sigma)
    D = gaussdx(x, sigma)
    g = np.outer(G, D)

    # compute Dx
    imgDx = convolve2d(img, g, mode='same')
    # compute Dy
    imgDy = convolve2d(img, np.transpose(g), mode='same')

    # quantize the images to "bins" number of values
    a = imgDx.min(); b = imgDx.max(); imgDx = 159/(b-a)*(imgDx-a)
    a = imgDy.min(); b = imgDy.max(); imgDy = 159/(b-a)*(imgDy-a)
    imgDx = np.floor(imgDx*(bins/160)) + 1
    imgDy = np.floor(imgDy*(bins/160)) + 1

    # define a 2D histogram with "bins^2" number of entries
    h = np.zeros((bins, bins))

    # execute the loop for each pixel in the image
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            # increment a histogram bin which corresponds to the value
            # of pixel i, j
            dx = int(imgDx[i, j])
            dy = int(imgDy[i, j])
            h[dx - 1, dy - 1] += 1

    # normalize the histogram such that its integral (sum) is equal 1
    h = h / np.sum(h)
    h = np.reshape(h, (bins ** 2, 1))
    return h

def descriptors_dxdy(img, px, py, m, sigma, bins):
    img = img.astype(float)
    rad = int((m-1)/2)
    h, w = img.shape
    D = np.zeros((len(px), bins**2))

    for i in range(len(px)):
        minx = max(px[i]-rad, 0)
        maxx = min(px[i]+rad+1, w)
        miny = max(py[i]-rad, 0)
        maxy = min(py[i]+rad+1, h)

        imgWin = img[miny:maxy, minx:maxx]
        hist = histdxdy(imgWin, sigma, bins)
        D[i, :] = hist.reshape(-1)

    return D

### Implement your pattern matching here:

In [ ]:
def findnn(D1, D2):
    #TODO:find nearest neighbour index and corresponding distance. hint:use "dist = np.linalg.norm(.)" function for computing euclidean distance.
    return Idx, Dist

### Work with your images here:

In [ ]:
# Read the images
img1 = cv2.imread('./images/ucla/img1.JPG')
img2 = cv2.imread('./images/ucla/img2.JPG')


sigma1 = 2.0
sigma2 = 1.6
threshold = 1000

# parameters for regional descriptors
m = 41
bins = 16

# estimate the Harris keypoints for first image
ImageRes1 = compute_harris(img1, sigma1, sigma2)
px1, py1 = non_max_suppression(ImageRes1, threshold)

# estimate the Harris keypoints for second image
ImageRes2 = compute_harris(img2, sigma1, sigma2)
px2, py2 = non_max_suppression(ImageRes2, threshold)

# regional descriptors
D1 = descriptors_rg(img1, px1, py1, m, bins)
D2 = descriptors_rg(img2, px2, py2, m, bins)

# You can try another descriptor here
# D1 = descriptors_dxdy(cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY), px1, py1, m, sigma2, bins)
# D2 = descriptors_dxdy(cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY), px2, py2, m, sigma2, bins)

# find best matching points using Euclidean distance
match_index, match_dist = findnn(D1, D2)

# visualization of matching
N = 8
displaymatches(img1, px1, py1, img2, px2, py2, match_index, match_dist, N)
